In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sS = pd.read_csv("sampleSubmission.csv")

In [3]:
select_label = [0, 1, 2, 3]
select_habits = [0., 1.]

In [4]:
train['推薦類型'].replace(['肌力',"柔軟度","心肺","肌耐力"],[0,1,2,3] , inplace = True)

In [5]:
train = train[train['推薦類型'].isin(select_label)]

In [6]:
train = train[train['拳擊'].isin(select_habits)]
train = train[train['器材'].isin(select_habits)]
train = train[train['有氧'].isin(select_habits)]
train = train[train['舞蹈'].isin(select_habits)]
train = train[train['高強度間歇'].isin(select_habits)]
train = train[train['瑜伽'].isin(select_habits)]
train = train[train['肌力訓練'].isin(select_habits)]
train = train[train['燃脂'].isin(select_habits)]
train = train[train['減脂瘦身'].isin(select_habits)]
train = train[train['增加肌肉'].isin(select_habits)]
train = train[train['增強體魄'].isin(select_habits)]
train = train[train['舒緩壓力'].isin(select_habits)]

In [7]:
train = train[train['年齡']<70]

In [8]:
train = train[train['BMI']>= 15]

In [9]:
train['推薦類型'] = train['推薦類型'].astype(int)

In [10]:
X = train.drop(["推薦類型"], axis =1)
y = train["推薦類型"]

In [11]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
X_s, y_s = SMOTE().fit_resample(X,y)
X_s, y_s = TomekLinks().fit_resample(X_s, y_s)

In [12]:
train = pd.concat([X_s,y_s], axis =1, join = 'outer')

In [13]:
train

,性別,年齡,BMI,拳擊,器材,有氧,舞蹈,高強度間歇,瑜伽,肌力訓練,燃脂,減脂瘦身,增加肌肉,增強體魄,舒緩壓力,推薦類型
0,1.0,47.000000,22.521372,0.000000,0.0,1.00000,1.0,1.00000,1.000000,0.0,1.000000,1.0,1.000000,1.000000,1.0,0
1,1.0,46.000000,23.161083,1.000000,1.0,1.00000,0.0,1.00000,0.000000,1.0,1.000000,0.0,1.000000,0.000000,1.0,0
2,1.0,43.000000,22.053277,0.000000,1.0,1.00000,1.0,1.00000,0.000000,0.0,0.000000,1.0,1.000000,1.000000,1.0,1
3,1.0,38.000000,19.310658,0.000000,1.0,1.00000,1.0,0.00000,0.000000,1.0,0.000000,1.0,0.000000,1.000000,1.0,0
4,1.0,45.000000,20.259939,1.000000,1.0,1.00000,1.0,1.00000,1.000000,1.0,1.000000,0.0,1.000000,0.000000,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,0.0,24.000000,16.618287,0.018495,1.0,0.00000,0.0,1.00000,1.000000,1.0,1.000000,0.0,0.981505,0.018495,0.0,3
4382,0.0,22.000000,16.961927,0.000000,1.0,0.52034,0.0,0.47966,1.000000,1.0,1.000000,0.0,1.000000,0.520340,0.0,3
4383,0.0,31.000000,19.208261,1.000000,0.0,0.00000,0.0,1.00000,1.000000,0.0,1.000000,0.0,0.000000,1.000000,1.0,3
4384,0.0,27.000000,15.275633,1.000000,0.0,0.00000,0.0,1.00000,0.790321,1.0,0.209679,0.0,1.000000,0.790321,0.0,3


In [14]:
train = train[train['拳擊'].isin(select_habits)]
train = train[train['器材'].isin(select_habits)]
train = train[train['有氧'].isin(select_habits)]
train = train[train['舞蹈'].isin(select_habits)]
train = train[train['高強度間歇'].isin(select_habits)]
train = train[train['瑜伽'].isin(select_habits)]
train = train[train['肌力訓練'].isin(select_habits)]
train = train[train['燃脂'].isin(select_habits)]
train = train[train['減脂瘦身'].isin(select_habits)]
train = train[train['增加肌肉'].isin(select_habits)]
train = train[train['增強體魄'].isin(select_habits)]
train = train[train['舒緩壓力'].isin(select_habits)]

In [15]:
train

,性別,年齡,BMI,拳擊,器材,有氧,舞蹈,高強度間歇,瑜伽,肌力訓練,燃脂,減脂瘦身,增加肌肉,增強體魄,舒緩壓力,推薦類型
0,1.0,47.000000,22.521372,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
1,1.0,46.000000,23.161083,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0
2,1.0,43.000000,22.053277,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1
3,1.0,38.000000,19.310658,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0
4,1.0,45.000000,20.259939,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378,1.0,27.000000,23.875115,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,3
4379,1.0,27.000000,23.875115,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,3
4380,1.0,26.481705,28.214887,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3
4383,0.0,31.000000,19.208261,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,3


In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV
X_train, X_val, y_train, y_val = train_test_split(
   X_s, y_s, test_size=0.33, random_state=42)


In [17]:
xg1 = XGBClassifier(n_estimators= 200, max_depth = 8)
xg1=xg1.fit(X_train, y_train)
print(xg1.score(X_train,y_train))
xg1.score(X_val,y_val)

C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:06:21] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9921715452688904


0.8729281767955801

In [18]:
from sklearn.ensemble import RandomForestClassifier
RF_param = {"n_estimators":[50,100,150,200,500,1000],"max_depth":[5,10,15,20],"max_features" : [6,8,10,12,14]}

In [19]:
'''
RF = RandomForestClassifier()
GSCV = GridSearchCV(RF,param_grid=RF_param,cv=5)
GSCV.fit(X_train,y_train)
print("accuracy score",GSCV.score(X_val,y_val))
print("The best model param :",GSCV.best_params_)'''

KeyboardInterrupt: 

In [19]:
RF = RandomForestClassifier()
RF = RF.fit(X_train,y_train)
print(xg1.score(X_train,y_train))
xg1.score(X_val,y_val)

0.9921715452688904


0.8729281767955801

In [20]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC


def accuracy_score(y_val,y_pred):
    Sum = 0
    y_val = y_val.tolist()
    y_pred = y_pred.tolist()
    for i in range(len(y_pred)):
        if y_val[i] == y_pred[i]:
            Sum+=1
    score = Sum/len(y_pred)
    
    return score

In [21]:
rnd_clf = RandomForestClassifier(random_state=0,n_estimators=150,max_features = 6)
#svm_clf = SVC(random_state=0)
xgb_clf =  XGBClassifier(random_state=0, n_estimators=400, max_depth = 8,learning_rate =0.1)

voting_clf = VotingClassifier(estimators=[('xg', xgb_clf), ('rf', rnd_clf)],
 voting='hard')

voting_clf.fit(X_train, y_train)

for clf in (xgb_clf, rnd_clf,voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))
    

C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:06:38] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:06:41] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier 0.8736187845303868
RandomForestClassifier 0.8839779005524862
[22:06:44] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


VotingClassifier 0.8784530386740331


In [ ]:
voting_clf.fit(X, y)
y_pred = RF.predict(test)
y_pred_pd = pd.DataFrame(data = y_pred, columns = ["label"])
y_pred_pd = y_pred_pd.astype('int64').reset_index()
y_pred_pd.to_csv("e94084032.csv", index = False)

In [ ]:
print(eee)###################one_hot

In [22]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categories = 'auto')
from sklearn.compose import ColumnTransformer
#Encode Country Column
ct = ColumnTransformer([("性別", OneHotEncoder(), [0])], remainder = 'passthrough')
train_one_hot = ct.fit_transform(train)
train_one_hot = pd.DataFrame(train_one_hot)

In [23]:
train_one_hot = train_one_hot.rename(columns = {0:'女',1:'男',2:'年齡',3:'BMI',4:'拳擊',5:'器材',6:'有氧',7:'舞蹈',8:'高強度間歇',9:'瑜伽',10:'肌力訓練',11:'燃脂',12:'減脂瘦身',13:'增加肌肉',14:'增強體魄',15:'舒緩壓力',16:'Label'})

In [24]:
train_f_one_hot = train_one_hot.drop(["Label"], axis =1)
print(train_f_one_hot.shape)
train_f_one_hot_y = train_one_hot["Label"]

(3858, 16)


In [25]:
X_train, X_val, y_train, y_val = train_test_split(
   train_f_one_hot, train_f_one_hot_y, test_size=0.33, random_state=42)

In [26]:
RF = RandomForestClassifier(random_state=0,n_estimators=2000)
RF = RF.fit(X_train,y_train)
print(RF.score(X_train,y_train))
RF.score(X_val,y_val)

0.9922600619195047


0.8626373626373627

In [27]:
xg2 = XGBClassifier(n_estimators= 200, max_depth = 8)
xg2=xg2.fit(X_train, y_train)
print(xg2.score(X_train,y_train))
xg2.score(X_val,y_val)

C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:07:31] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9922600619195047


0.8367346938775511

In [28]:
rnd_clf = RandomForestClassifier(random_state=0,n_estimators=200)
#svm_clf = SVC(random_state=0)
xgb_clf =  XGBClassifier(random_state=0, n_estimators=200, max_depth = 8,learning_rate =0.1,min_child_weight=1)

voting_clf = VotingClassifier(estimators=[('xg', xgb_clf), ('rf', rnd_clf)],
 voting='hard')

voting_clf.fit(X_train, y_train)

for clf in (xgb_clf, rnd_clf,voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))

C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:07:53] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:07:55] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier 0.8359497645211931
RandomForestClassifier 0.8579277864992151
[22:07:57] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\KYLE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


VotingClassifier 0.847723704866562


In [ ]:
ct = ColumnTransformer([("性別", OneHotEncoder(), [0])], remainder = 'passthrough')
test_one_hot = ct.fit_transform(test)

test_one_hot = pd.DataFrame(test_one_hot)

test_one_hot= test_one_hot.rename(columns = {0:'女',1:'男',2:'年齡',3:'BMI',4:'拳擊',5:'器材',6:'有氧',7:'舞蹈',8:'高強度間歇',9:'瑜伽',10:'肌力訓練',11:'燃脂',12:'減脂瘦身',13:'增加肌肉',14:'增強體魄',15:'舒緩壓力'})

RF.fit(train_f_one_hot, train_f_one_hot_y)
y_pred = RF.predict(test_one_hot)
y_pred_pd = pd.DataFrame(data = y_pred, columns = ["label"])
y_pred_pd = y_pred_pd.astype('int64').reset_index()
y_pred_pd.to_csv("e94084032.csv", index = False)

In [1]:
##